참조 코드: https://developer-woo.tistory.com/60

### 천안시 시설 및 프로그램 리뷰 크롤링

In [1]:
import re
import time
import json
import urllib.request
from tqdm import tqdm  
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

# Naver API key 가져오기
load_dotenv()
CLIENT_ID = os.getenv('client_id')
CLIENT_SECRET = os.getenv('client_secret')

In [2]:
def blog_search(keyword, end_page, crawling_page_num,client_id, client_secret):
    # 검색어 입력
    keyword = keyword
    # Selenium으로 검색 페이지 불러오기
    naver_data = []
    encText = urllib.parse.quote(keyword)

    # 검색을 끝낼 페이지 입력
    # end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
    end = end_page
    if end == "":
        end = 1
    else:
        end = int(end)
    print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행합니다")

    # 한번에 가져올 페이지 입력
    #display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
    display = crawling_page_num
    if display == "":
        display = 10
    else:
        display = int(display)
    print("\n한번에 가져올 페이지 : ", display, "페이지")

    for start in range(end):
        url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start+1}&display={display+1}" # JSON 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        
        if rescode == 200:
            response_body = response.read()
            data = json.loads(response_body.decode('utf-8'))['items']
            
            for row in data:
                if 'blog.naver' in row['link']:
                    # HTML 태그 제거
                    title = re.sub('<[^>]*>', '', row['title'])
                    if title == '전국의 가을여행 명소':
                        pass
                    url = row['link']
                    post_date = row['postdate']
                    
                    # JSON 데이터 추가
                    naver_data.append({
                        "title": title,
                        "url": url,
                        "postdate": post_date,
                    })
            time.sleep(2)
        else:
            print("Error Code:" + rescode)
    return naver_data

In [3]:
def blog_contents(url):

    # requests를 사용하여 페이지 가져오기
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # blog 게시물 내용은 ififrame을 찾기
    iframe = soup.find('iframe', id='mainFrame')

    # iframe의 src 속성 가져오기
    iframe_url = 'https://blog.naver.com' + iframe['src']

    # iframe의 실제 페이지 가져오기
    iframe_response = requests.get(iframe_url)
    iframe_content = iframe_response.text

    # 다시 iframe의 내용 파싱
    iframe_soup = BeautifulSoup(iframe_content, 'html.parser')

    # 블로그 본문 추출
    content_divs = iframe_soup.find_all('div', class_='se-component se-text se-l-default')
    text = ""
    # 각 div에서 텍스트를 추출하여 출력
    for div in content_divs:
        paragraphs = div.find_all('p')
        for p in paragraphs:
            text += p.get_text(strip=True) + " "  # 각 문단을 공백으로 연결

    return text

In [4]:
keywords = ['도솔광장 반려인 쉼터', '천안축구센터 반려인 쉼터', '백석동 반려인 쉼터','천안시 동물보호센터','2024 천안시 dog festival','천안시 펫티켓 문화교육']

In [6]:
blog = {}
end_page = 10
crawling_page_num = 50

for keyword in keywords: 
    
    # Naver API를 사용하여 검색 결과 가져오기
    data = blog_search(keyword, end_page, crawling_page_num, CLIENT_ID, CLIENT_SECRET)

    for i in tqdm(range(len(data))):
        url = data[i]['url']
        text = blog_contents(url)
        data[i]['text'] = text

    # 첫 번째 키워드를 blog 딕셔너리에 추가
    if keyword not in blog:
        blog[keyword] = data  # 첫 번째 키워드의 데이터를 추가

    else:
        blog[keyword].extend(data)  # 기존 키워드에 새로운 데이터를 추가


 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 40/40 [00:10<00:00,  3.97it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 235/235 [00:54<00:00,  4.28it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


...

100%|██████████| 510/510 [01:37<00:00,  5.24it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 9/9 [00:02<00:00,  4.18it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 125/125 [00:32<00:00,  3.90it/s]


In [9]:
blog['도솔광장 반려인 쉼터'][0]

{'title': '천안 애견동반 가능한 도솔광장 ㅣ 반려인 쉼터와 나라꽃... ',
 'url': 'https://blog.naver.com/requiem_09/223477750366',
 'postdate': '20240613',
 'text': '콩이를 데리고 천안가볼만한곳 을 물색하던 도중 도솔광장이 좋다는 소리에 달려온 양구콩 양콩이가 소개해준 콩이 시리즈! 그 첫 번째#도솔광장#도솔광장반려인쉼터 이렇게 넓은 지하주차장에 저렴한 주차요금 그리고 2시간 무료라니?! 우선 합격~! 반려동물 펫티켓은 반드시 지켜져야하고 우리 콩이도 안전한 산책을 하려면 모든 견주들이 펫티켓을 지켜줘야 한다는 사실! 클라이밍 장소도 있고 화장실도 곳곳에 있어서 여러가지 액티비티를 즐기기에도 좋아요 단, 클라이밍은 사전에 허가받고 정해진 시간에만 이용이 가능한가봐요 저희의 목적지로 가는 길에 오잉?! 나라꽃 무궁화숲길이 있더라구요 #무궁화숲길#천안무궁화숲길 잘은 모르지만 천안시에서 나라꽃을 심어주고 관리해주고 있다는 사실은 확실하네요 이렇게 무궁화에 대한 여러 정보도 공유하면서 여러가지 볼거리를 만들어주니 사람들이 안올 수가 없죠~! 으음 무궁화도 한 종류가 아니라 여러 구분을 통해 분류가 되는구나를 느낀 구콩이 천연기념물인 무궁화에 대해 궁금하시다면 도솔광장을 방문해보세요~! 이 와중에 네잎클로버 찾기에서 정말 찾아버린 몽골의 피가 0.0001%이상 흐르는 양콩이! 다음 날, 코팅해서 고이 간직했다는 양콩이 아유 순수해라~~ 사실 오늘의 주 목적지는 우리 콩이의 자유로운 뜀박질을 위한 공간이 있는 도솔광장 반려인쉼터에요 오자마자 그 존재감을 뽐내는 우리 콩이 친구들에게 가서 킁킁 냄새를 맡고 부담스럽게 접근해서 엄청 뛰어댕겼어요 여기저기 펄쩍거리는 콩이를 보면서 반려견주분들이 아이고 애기네~ 하셨지요 \u200b 우리 콩이 이제 14개월이라 애기 티는 벗어야 할텐데 저렇게 최선을 다해 뛰어다니면 누가 어른이라고 하겠어요,,, ㅠ 한 번 꽂힌 강아지는 졸졸졸 따라다니는 스토커

In [10]:
# JSON 파일로 저장
with open('Data/blog.json', 'w', encoding='utf-8') as json_file:
    json.dump(blog, json_file, ensure_ascii=False, indent=4)

### 사립 시설 크롤링

In [14]:
keywords = ['천안 운동장 애견카페']

In [15]:
private_facil = {}
end_page = 10
crawling_page_num = 50

for keyword in keywords: 
    
    # Naver API를 사용하여 검색 결과 가져오기
    data = blog_search(keyword, end_page, crawling_page_num, CLIENT_ID, CLIENT_SECRET)

    for i in tqdm(range(len(data))):
        url = data[i]['url']
        text = blog_contents(url)
        data[i]['text'] = text

    # 첫 번째 키워드를 blog 딕셔너리에 추가
    if keyword not in blog:
        private_facil[keyword] = data  # 첫 번째 키워드의 데이터를 추가

    else:
        private_facil[keyword].extend(data)  # 기존 키워드에 새로운 데이터를 추가


 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 510/510 [02:20<00:00,  3.63it/s]


In [17]:
private_facil['천안 운동장 애견카페'][0]

{'title': '천안애견카페 포토존에 강아지운동장 있는 날아라멍멍',
 'url': 'https://blog.naver.com/kskr3105/223430968747',
 'postdate': '20240429',
 'text': '\u200b \u200b 오랜만에강아지운동장에서 뛰뛰시키고 싶어서천안애견카페날아라멍멍을 찾았어요. 주소상은 아산 탕정이지만 천안 불당동과 10분 거리밖에 되지 않아서 아주 가까웠어요. 직접 방문해 봤더니 블루 도어의 외관이 눈길을 사로잡았어요. 외국 감성이 뿜뿜하는 카페랄까요? 애카가 아니라 일반 갬성 카페 같은 분위기였어요. 바로 앞에넓은 단독 주차공간이 있어서 오시는 분들은 주차 걱정 안 해도 될 것 같아요. \u200b \u200b \u200b 알고 보니tv 동물농장에도 나왔던 곳이었어요. 아무튼 간에 입장하기 전 이용수칙이 있어서 열심히 읽어준 후 입장했어요. 우선, 반군이 애들을 데리고 강아지운동장으로 바로 입장했고 저 혼자만 실내로 들어갔어요. 주차장에서 운동장으로 들어가는 문이 있어서 편하게 바로 입장할 수 있었어요.날아라멍멍 영업시간은 평일은 오전 11시 ~ 오후 7시까지, 주말은 오후 8시까지이며 매주 월요일이 휴무이니 참고하셔서 방문하면 좋을 것 같아요. 저희는 낑깡이땜에 오픈 시간에 맞춰서 오전 11시에 방문했어요. \u200b \u200b \u200b 문을 열고 들어가니 안전문으로 한 번 더 있어서 실내 카페에서 놀다가 아이가 뛰쳐나가는 일은 없을듯해요. 그리고 실내화가 있어서 순간 신발을 벗는 건가 했는데 여쭤보니 비가 온 날에만 신는다고 하더라고요. 얼마나깔끔하게 관리를 하는지 이때부터 느낄 수 있었어요. \u200b \u200b \u200b 들어오자마자 저도 모르게 감탄이 절로 나왔어요. 사실 밖에서 봤을 땐 강아지운동장은 넓어도 실내는 아담한 천안애견카페라고 생각했어요. 근데약 50평 정도 되는 실내였으며 무엇보다 너무 깨끗했어요. 아무리 열자마자 왔다고 해도 제가 지금까지 직접 다녀봤던 애카들은 애

In [18]:
# JSON 파일로 저장
with open('Data/private_facil_blog.json', 'w', encoding='utf-8') as json_file:
    json.dump(private_facil, json_file, ensure_ascii=False, indent=4)